# Virtual Try-On: Basic Use

All virtual try-on operations require three inputs (source image, reference image, mask) and produce 1 output (generated Image). The mask defines which area will be modified in the output.

- The area inside the mask: will be modified (“inpainted”) using the reference image as guide for what to fill
- The area outside the mask: will be preserved as-is

VTO provides three different masking modes via the `maskType` parameter:

`“GARMENT”` is an automatic masking mode in which a mask is generated on your behalf for an area of the body. When using the “GARMENT” mask type, you will also set a `garmentBasedMask.garmentClass` value to indicate which body area in the source image should be masked.

`“PROMPT”` is another automatic masking mode, particularly useful for non-garment use cases like furniture and other products. When using the “PROMPT” mask type, you will set a `promptBasedMask.maskPrompt` value describing the area of the source image to be changed. This is identical to how the maskPrompt parameter of the Nova Canvas “INPAINTING” and “OUTPAINTING” task types work.

`“IMAGE”` allows you to provide your own black and white image to be used as a mask, where black indicates the pixels of the source image to change and white indicates the pixels that should be unaffected. Use an image mask to add accessories to a person (handbag, jewlery, hat, etc.); to place products, logos, or other visual elements; or in garment scenarios where you want more control over styling of a garment.

#### "GARMENT" Mask Example

This example demonstrates the minimum configuration needed for a garment try-on scenario. The reference image can contain a garment on or off body. The model is intelligent enough to only transfer the garment(s) from the reference image.

Nova Canvas uses a black and white image mask to determine which part of the source image should be modified. Users can provide this image themselves, but the model also supports auto-masking modes that eliminate the need to provide your own mask image.

In this example, we'll use a `maskType` of `"GARMENT"` which is one of the automatic masking modes. You simply indicate which type of garment you are trying to visualize via the `garmentClass` parameter, and Nova Canvas will take care of the rest. Refer the the [API Guide](https://quip-amazon.com/iwMyAbIqS8bS/2-Nova-Canvas-Refresh-Beta-API-Guide) for a full list of supported `garmentClass` values.

Run the cell below, then experiment with your own images.

<div style="display: flex; gap: 2em; margin-right: 2em">
  <div style="text-align: center; flex: 1">
    <img src="../images/man-upper_body-sweater.png">
    <strong><tt>sourceImage</tt></strong>
  </div>
  <div style="text-align: center; flex: 1">
    <img src="../images/jersey-long_sleeve.png">
    <strong><tt>referenceImage</tt></strong>
  </div>
   <div style="text-align: center; flex: 1">
    <img src="../images/top_output.png">
    <strong>Example output</strong>
  </div>
</div>


In [ ]:
import logging
import os
from datetime import datetime
from random import randint
from amazon_nova_canvas_utils import (
    generate_images,
    load_image_as_base64,
    base64_to_pil_image,
)

logging.basicConfig(format="[%(levelname)s] %(message)s", level=logging.INFO)

# Edit these values to experiment with your own images.
source_image_path = "../images/vto-images/vto_garment_mask_source.jpg"
reference_image_path = "../images/vto-images/vto_garment_mask_reference.jpg"

inference_params = {
    "taskType": "VIRTUAL_TRY_ON",
    "virtualTryOnParams": {
        "sourceImage": load_image_as_base64(source_image_path),
        "referenceImage": load_image_as_base64(reference_image_path),
        "maskType": "GARMENT",
        "garmentBasedMask": {"garmentClass": "UPPER_BODY"},
    },
    # The following is optional but provided here for you to experiment with.
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "cfgScale": 6.5,
        "seed": randint(0, 2147483646),
    },
}

output_folder = os.path.join("output", datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

try:
    response_body = generate_images(
        inference_params=inference_params,
        save_folder_path=output_folder,
        model_id="amazon.nova-canvas-v1:0",
        region_name="us-east-1",
    )

    # An error message may be returned, even if some images were generated.
    if "error" in response_body:
        logging.error(response_body["error"])

    if "images" in response_body:
        # Display all images.
        for image_base64 in response_body["images"]:
            image = base64_to_pil_image(image_base64)
            display(image)

except Exception as e:
    logging.error(e)

print(f"Done! Artifacts saved to {os.path.abspath(output_folder)}")